### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = English
Lang Id = 1
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [9]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,for what it is that we do,1323,1325,for what it is that we do,QoTjKrw2VUI,https://www.youtube.com/watch?v=QoTjKrw2VUI&t=...
1,what it is this is the,74,76,so do you know what it is this is the honda,H6E6N70jYqI,https://www.youtube.com/watch?v=H6E6N70jYqI&t=74s
2,i will do this this is,140,143,i will do this this is going to happen,Fc80L8NNBgQ,https://www.youtube.com/watch?v=Fc80L8NNBgQ&t=...
3,do this i will do this,160,163,i must do this i will do this,Fc80L8NNBgQ,https://www.youtube.com/watch?v=Fc80L8NNBgQ&t=...
4,this on and i have a,397,400,leave this on and i have a gun or not a,M5hEQRvIYBI,https://www.youtube.com/watch?v=M5hEQRvIYBI&t=...
...,...,...,...,...,...,...
195,what is my,61,63,what is my name,iXlEwBfwG2I,https://www.youtube.com/watch?v=iXlEwBfwG2I&t=61s
196,this is it,1004,1006,this is it man,iWaO5II3p2w,https://www.youtube.com/watch?v=iWaO5II3p2w&t=...
197,what is it,138,139,what is it,pBk4_XNCHJQ,https://www.youtube.com/watch?v=pBk4_XNCHJQ&t=...
198,to do that,1125,1126,and so to do that,2Zt9oJtFKGw,https://www.youtube.com/watch?v=2Zt9oJtFKGw&t=...


In [10]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,it but yeah then that you get just a little as...,113,122,it but yeah then that you get just a little as...,9xGdqXewfag,https://www.youtube.com/watch?v=9xGdqXewfag&t=...
1,take a look at or just something you believe i...,1106,1111,take a look at or just something you believe i...,wHsCWUXHYh4,https://www.youtube.com/watch?v=wHsCWUXHYh4&t=...
2,tell you where we were or where we were going ...,843,846,i couldnt tell you where we were or where we w...,YHiArH94KtI,https://www.youtube.com/watch?v=YHiArH94KtI&t=...
3,years to come and you know you can come back a...,119,130,be there for many years to come and you know y...,Z8ezoUYzgQ0,https://www.youtube.com/watch?v=Z8ezoUYzgQ0&t=...
4,what do you do what can you do i mean it would...,3236,3243,what do you do what can you do i mean it would...,YHiArH94KtI,https://www.youtube.com/watch?v=YHiArH94KtI&t=...
...,...,...,...,...,...,...
195,that some of the people who are maybe more,156,159,i found that some of the people who are maybe ...,JIGABm9crzk,https://www.youtube.com/watch?v=JIGABm9crzk&t=...
196,not really only this oh yeah sorry yeah my,1618,1622,not really only this oh yeah sorry yeah my bad,D0cp0bNXYjs,https://www.youtube.com/watch?v=D0cp0bNXYjs&t=...
197,go have a look around and see what this on,2274,2277,go have a look around and see what this on off...,400mu7HvssI,https://www.youtube.com/watch?v=400mu7HvssI&t=...
198,and just there was something about the way,645,648,and just there was something about the way,QoTjKrw2VUI,https://www.youtube.com/watch?v=QoTjKrw2VUI&t=...


In [11]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,now this shot could have been the top shot of ...,199,206,now this shot could have been the top shot of ...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
1,like get your stuff together away your stuff c...,123,131,like get your stuff together away your stuff c...,5UMN_CpEwKY,https://www.youtube.com/watch?v=5UMN_CpEwKY&t=...
2,shoot with so this is gon na happen all the ti...,14,21,shoot with so this is gon na happen all the ti...,4gfL5Ta3dMI,https://www.youtube.com/watch?v=4gfL5Ta3dMI&t=14s
3,we love when you are here with us um if you wo...,655,662,we love when you are here with us um if you wo...,__jg98sdg_Q,https://www.youtube.com/watch?v=__jg98sdg_Q&t=...
4,maybe do i got nothing to lose i might just tr...,528,536,maybe do i got nothing to lose i might just tr...,WTvQx4qqjxM,https://www.youtube.com/watch?v=WTvQx4qqjxM&t=...
...,...,...,...,...,...,...
195,pretty much right after that happened it must ...,2260,2263,pretty much right after that happened it must ...,YHiArH94KtI,https://www.youtube.com/watch?v=YHiArH94KtI&t=...
196,see where we came from not good at that they g...,1644,1654,see where we came from not good at that they g...,8QaOxzQTNcQ,https://www.youtube.com/watch?v=8QaOxzQTNcQ&t=...
197,that would have ran along this very street tha...,101,107,that would have ran along this very street tha...,C_hc3KJXvak,https://www.youtube.com/watch?v=C_hc3KJXvak&t=...
198,oh yeah some people might not been here last w...,771,775,oh yeah some people might not been here last w...,sim3KsItki0,https://www.youtube.com/watch?v=sim3KsItki0&t=...


In [12]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,now this shot could have been the top shot of ...,199,206,now this shot could have been the top shot of ...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
1,so will the prime minister take time this chri...,23,34,so will the prime minister take time this chri...,evorzjZYtzU,https://www.youtube.com/watch?v=evorzjZYtzU&t=23s
2,we crossed i still hear your voice in the traf...,218,245,sidewalks we crossed i still hear your voice i...,3oBRcjLxQ4c,https://www.youtube.com/watch?v=3oBRcjLxQ4c&t=...
3,solve your problems must not have had enough m...,72,83,whoever said money cant solve your problems mu...,XAeRKX6BrFk,https://www.youtube.com/watch?v=XAeRKX6BrFk&t=72s
4,a number of stops on route today are request s...,432,441,a number of stops on route today are request s...,WnmkHR11bfg,https://www.youtube.com/watch?v=WnmkHR11bfg&t=...
...,...,...,...,...,...,...
195,is not clear why yet um uh its clearly ridicul...,1276,1285,is not clear why yet um uh its clearly ridicul...,TcXDS8VKB9A,https://www.youtube.com/watch?v=TcXDS8VKB9A&t=...
196,pussy here girl take my whole credit card ill ...,52,60,mediocre pussy here girl take my whole credit ...,xq3BYneq9Ec,https://www.youtube.com/watch?v=xq3BYneq9Ec&t=52s
197,coming from not just from what they say but fr...,1451,1457,coming from not just from what they say but fr...,MvJH1w5zNN4,https://www.youtube.com/watch?v=MvJH1w5zNN4&t=...
198,yeah yeah it makes sense so for uh question nu...,606,612,yeah yeah it makes sense so for uh question nu...,wHsCWUXHYh4,https://www.youtube.com/watch?v=wHsCWUXHYh4&t=...


In [13]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,a toilet can be found in each coach and everyt...,477,486,a toilet can be found in each coach and everyt...,WnmkHR11bfg,https://www.youtube.com/watch?v=WnmkHR11bfg&t=...
1,red lights stop signs i still see your face in...,190,217,red lights stop signs i still see your face in...,3oBRcjLxQ4c,https://www.youtube.com/watch?v=3oBRcjLxQ4c&t=...
2,we crossed i still hear your voice in the traf...,218,245,sidewalks we crossed i still hear your voice i...,3oBRcjLxQ4c,https://www.youtube.com/watch?v=3oBRcjLxQ4c&t=...
3,nevertheless i love this shot there is a myste...,206,215,nevertheless i love this shot there is a myste...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
4,a number of stops on route today are request s...,432,441,a number of stops on route today are request s...,WnmkHR11bfg,https://www.youtube.com/watch?v=WnmkHR11bfg&t=...
...,...,...,...,...,...,...
195,um but were actually based virtually technical...,212,218,um but were actually based virtually technical...,wKvjuhIWDvk,https://www.youtube.com/watch?v=wKvjuhIWDvk&t=...
196,two systems of bringing it in they either brin...,200,206,theres two systems of bringing it in they eith...,fRbZw_c6Vyc,https://www.youtube.com/watch?v=fRbZw_c6Vyc&t=...
197,its an unlikely bunch of people but it does wo...,543,548,its an unlikely bunch of people but it does wo...,CucE4cF7Ris,https://www.youtube.com/watch?v=CucE4cF7Ris&t=...
198,they may not always be considered growth stock...,35,41,they may not always be considered growth stock...,nBSUfb61dR8,https://www.youtube.com/watch?v=nBSUfb61dR8&t=35s


In [14]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,on that server which has been missed the progr...,164,176,on that server which has been missed the progr...,GJ1ofG_O5uY,https://www.youtube.com/watch?v=GJ1ofG_O5uY&t=...
1,are there some aces up your sleeve have you no...,23,39,are there some aces up your sleeve have you no...,DrfEdIolHQg,https://www.youtube.com/watch?v=DrfEdIolHQg&t=23s
2,nevertheless i love this shot there is a myste...,206,215,nevertheless i love this shot there is a myste...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
3,well if youre persistent enough and you go aft...,168,177,well if youre persistent enough and you go aft...,c4NiXJZC6Ck,https://www.youtube.com/watch?v=c4NiXJZC6Ck&t=...
4,another couple of things i unfortunately didnt...,516,524,another couple of things i unfortunately didnt...,WnmkHR11bfg,https://www.youtube.com/watch?v=WnmkHR11bfg&t=...
...,...,...,...,...,...,...
195,gained another stand member this is martin hel...,565,572,gained another stand member this is martin hel...,X9gl09ScRyM,https://www.youtube.com/watch?v=X9gl09ScRyM&t=...
196,theyre all pulling together very effectively b...,59,65,theyre all pulling together very effectively b...,sHeOWzq3GFk,https://www.youtube.com/watch?v=sHeOWzq3GFk&t=59s
197,you in for a closer look shortly okay lets blo...,413,425,you in for a closer look shortly okay lets blo...,AfczPeLFi1Q,https://www.youtube.com/watch?v=AfczPeLFi1Q&t=...
198,um they are very good in hiding their identity...,486,494,um they are very good in hiding their identity...,MvJH1w5zNN4,https://www.youtube.com/watch?v=MvJH1w5zNN4&t=...


In [15]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,been wonderin if your hearts still open and if...,104,128,been wonderin if your hearts still open and if...,DrfEdIolHQg,https://www.youtube.com/watch?v=DrfEdIolHQg&t=...
1,on that server which has been missed the progr...,164,176,on that server which has been missed the progr...,GJ1ofG_O5uY,https://www.youtube.com/watch?v=GJ1ofG_O5uY&t=...
2,for example banks that calculate interest rate...,566,577,for example banks that calculate interest rate...,GJ1ofG_O5uY,https://www.youtube.com/watch?v=GJ1ofG_O5uY&t=...
3,are there some aces up your sleeve have you no...,23,39,are there some aces up your sleeve have you no...,DrfEdIolHQg,https://www.youtube.com/watch?v=DrfEdIolHQg&t=23s
4,nevertheless i love this shot there is a myste...,206,215,nevertheless i love this shot there is a myste...,WTBUdOIqOVs,https://www.youtube.com/watch?v=WTBUdOIqOVs&t=...
...,...,...,...,...,...,...
195,led lights so you can see all your clothes thi...,326,334,led lights so you can see all your clothes thi...,e6cCvY3Pr1Y,https://www.youtube.com/watch?v=e6cCvY3Pr1Y&t=...
196,are collagen supplements a scam or are they st...,1040,1048,are collagen supplements a scam or are they st...,Wjqe5_ZHGz0,https://www.youtube.com/watch?v=Wjqe5_ZHGz0&t=...
197,or adult fantasy it is written by an author th...,660,668,or adult fantasy it is written by an author th...,OU2W0a2jllU,https://www.youtube.com/watch?v=OU2W0a2jllU&t=...
198,in gas prices has thus increased oil demand as...,133,141,in gas prices has thus increased oil demand as...,DjNCCpSPijY,https://www.youtube.com/watch?v=DjNCCpSPijY&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [16]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [17]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [18]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [19]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [20]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [21]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [22]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [23]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [24]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [25]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [26]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [27]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [28]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [29]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [30]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [31]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [32]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [33]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [34]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [35]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [36]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [37]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [38]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [39]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [40]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [41]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [42]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [43]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [44]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [45]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [46]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1671439469
  nanos: 250308000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass